### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/data/20220507_P183_octopus_SYTOXG/Atlas/P183_536BP_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/data/20220507_P183_octopus_SYTOXG/Atlas/mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "P183_536BP_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

aff = np.array([[ 1.0, 0, 0],
                [ 0, 1.0, 0],
                [ 0, 0, 1.0]], dtype='float')
transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/220617_P201_skeletonization/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,4",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=12)
R.set_fixed_image(os.path.join(tmpdir, "P183_536BP_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/220617_P201_skeletonization/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/220617_P201_skeletonization/ants/F2M_,/flash/ReiterU/tmp/220617_P201_skeletonization/ants/Warped.nii.gz,/flash/ReiterU/tmp/220617_P201_skeletonization/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/220617_P201_skeletonization/ants/P183_536BP_10um_masked.nii,/flash/ReiterU/tmp/220617_P201_skeletonization/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/220617_P201_skeletonization/ants/P183_536BP_10um_masked.nii,/flash/ReiterU/tmp/

DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST
 2DIAGNOSTIC,     1, -1.084638953064e-01, 1.797693134862e+308, 2.5169e+00, 2.5169e+00,
 2DIAGNOSTIC,     2, -1.084912420202e-01, 1.797693134862e+308, 2.5644e+00, 4.7472e-02,
 2DIAGNOSTIC,     3, -1.085392290316e-01, 1.797693134862e+308, 2.6141e+00, 4.9741e-02,
 2DIAGNOSTIC,     4, -1.086206360378e-01, 1.797693134862e+308, 2.6642e+00, 5.0052e-02,
 2DIAGNOSTIC,     5, -1.087272453600e-01, 1.797693134862e+308, 2.7142e+00, 4.9975e-02,
 2DIAGNOSTIC,     6, -1.088957365306e-01, 1.797693134862e+308, 2.7642e+00, 5.0063e-02,
 2DIAGNOSTIC,     7, -1.091302180852e-01, 1.797693134862e+308, 2.8140e+00, 4.9763e-02,
 2DIAGNOSTIC,     8, -1.092192368219e-01, 1.797693134862e+308, 2.8639e+00, 4.9903e-02,
 2DIAGNOSTIC,     9, -1.101623671205e-01, 1.797693134862e+308, 2.9138e+00, 4.9960e-02,
 2DIAGNOSTIC,    10, -1.134066287560e-01, 1.797693134862e+308, 2.9642e+00, 5.0367e-02,
 2DIAGNOSTIC,    11, -1.164251808059e-01, 1.7976

 2DIAGNOSTIC,     6, -3.669505467868e-01, 1.797693134862e+308, 1.3095e+01, 3.7064e-01,
 2DIAGNOSTIC,     7, -3.670812422246e-01, 1.797693134862e+308, 1.3452e+01, 3.5721e-01,
 2DIAGNOSTIC,     8, -3.672055387694e-01, 1.797693134862e+308, 1.3780e+01, 3.2790e-01,
 2DIAGNOSTIC,     9, -3.673045321836e-01, 1.797693134862e+308, 1.4119e+01, 3.3904e-01,
 2DIAGNOSTIC,    10, -3.676919894188e-01, 1.797693134862e+308, 1.4448e+01, 3.2887e-01,
 2DIAGNOSTIC,    11, -3.681104689758e-01, 1.797693134862e+308, 1.4764e+01, 3.1534e-01,
 2DIAGNOSTIC,    12, -3.687241348424e-01, 1.797693134862e+308, 1.5145e+01, 3.8127e-01,
 2DIAGNOSTIC,    13, -3.699318944194e-01, 1.797693134862e+308, 1.5513e+01, 3.6848e-01,
 2DIAGNOSTIC,    14, -3.708166883396e-01, 1.797693134862e+308, 1.5933e+01, 4.1980e-01,
 2DIAGNOSTIC,    15, -3.712404414947e-01, 4.655606877256e-04, 1.6258e+01, 3.2493e-01,
 2DIAGNOSTIC,    16, -3.729684817832e-01, 5.703421193489e-04, 1.6581e+01, 3.2277e-01,
 2DIAGNOSTIC,    17, -3.752312623705e-01, 7.0

 2DIAGNOSTIC,   102, -4.712953726079e-01, 2.058642365251e-04, 4.6348e+01, 2.9350e-01,
 2DIAGNOSTIC,   103, -4.733379893861e-01, 2.066649429942e-04, 4.6642e+01, 2.9470e-01,
 2DIAGNOSTIC,   104, -4.835277105103e-01, 2.515147533761e-04, 4.6977e+01, 3.3411e-01,
 2DIAGNOSTIC,   105, -4.924077098358e-01, 3.247950576516e-04, 4.7311e+01, 3.3446e-01,
 2DIAGNOSTIC,   106, -5.263429069662e-01, 5.265909998472e-04, 4.7598e+01, 2.8711e-01,
 2DIAGNOSTIC,   107, -5.301370858600e-01, 7.131643726911e-04, 4.8005e+01, 4.0642e-01,
 2DIAGNOSTIC,   108, -5.315169027842e-01, 8.713826289729e-04, 4.8325e+01, 3.1999e-01,
 2DIAGNOSTIC,   109, -5.323517815204e-01, 9.980677997353e-04, 4.8688e+01, 3.6359e-01,
 2DIAGNOSTIC,   110, -5.327968136741e-01, 1.083929083897e-03, 4.8999e+01, 3.1131e-01,
 2DIAGNOSTIC,   111, -5.330882728620e-01, 1.128182497179e-03, 4.9406e+01, 4.0613e-01,
 2DIAGNOSTIC,   112, -5.331358095464e-01, 1.138552247557e-03, 4.9722e+01, 3.1631e-01,
 2DIAGNOSTIC,   113, -5.333313799438e-01, 1.1003945356

 1DIAGNOSTIC,    13, -8.418996081797e-01, 2.286918602427e-03, 1.4569e+01, 1.0420e+00,
 1DIAGNOSTIC,    14, -8.444829482103e-01, 1.965261422500e-03, 1.5676e+01, 1.1073e+00,
 1DIAGNOSTIC,    15, -8.468046321150e-01, 1.689222891769e-03, 1.6788e+01, 1.1117e+00,
 1DIAGNOSTIC,    16, -8.490066221134e-01, 1.449637345435e-03, 1.7899e+01, 1.1107e+00,
 1DIAGNOSTIC,    17, -8.510279728026e-01, 1.235924792967e-03, 1.8995e+01, 1.0965e+00,
 1DIAGNOSTIC,    18, -8.529399286566e-01, 1.051516902253e-03, 2.0090e+01, 1.0950e+00,
 1DIAGNOSTIC,    19, -8.546826132525e-01, 9.003128983424e-04, 2.1150e+01, 1.0597e+00,
 1DIAGNOSTIC,    20, -8.563892231864e-01, 7.808037681832e-04, 2.2092e+01, 9.4208e-01,
 1DIAGNOSTIC,    21, -8.580091299338e-01, 6.857470630926e-04, 2.3078e+01, 9.8656e-01,
 1DIAGNOSTIC,    22, -8.595251263889e-01, 6.094187213887e-04, 2.4190e+01, 1.1120e+00,
 1DIAGNOSTIC,    23, -8.609388032128e-01, 5.455821933079e-04, 2.5323e+01, 1.1321e+00,
 1DIAGNOSTIC,    24, -8.623167807590e-01, 4.9185397222

 1DIAGNOSTIC,   109, -8.848862332690e-01, 2.564957963747e-06, 1.1135e+02, 1.1397e+00,
 1DIAGNOSTIC,   110, -8.849176664256e-01, 2.456615308082e-06, 1.1241e+02, 1.0533e+00,
 1DIAGNOSTIC,   111, -8.849505844009e-01, 2.377192743059e-06, 1.1332e+02, 9.1277e-01,
 1DIAGNOSTIC,   112, -8.849865352624e-01, 2.319619775350e-06, 1.1423e+02, 9.1264e-01,
 1DIAGNOSTIC,   113, -8.850059911812e-01, 2.206835903160e-06, 1.1514e+02, 9.1184e-01,
 1DIAGNOSTIC,   114, -8.850356220949e-01, 2.110228485185e-06, 1.1611e+02, 9.6190e-01,
 1DIAGNOSTIC,   115, -8.850648321070e-01, 2.051261772945e-06, 1.1725e+02, 1.1467e+00,
 1DIAGNOSTIC,   116, -8.850942993111e-01, 1.995641695294e-06, 1.1837e+02, 1.1179e+00,
 1DIAGNOSTIC,   117, -8.851224424263e-01, 1.935775829574e-06, 1.1947e+02, 1.0962e+00,
 1DIAGNOSTIC,   118, -8.851561409396e-01, 1.908558146746e-06, 1.2052e+02, 1.0539e+00,
 1DIAGNOSTIC,   119, -8.851825044648e-01, 1.877826053690e-06, 1.2165e+02, 1.1311e+00,
 1DIAGNOSTIC,   120, -8.852009860866e-01, 1.8253479197

 1DIAGNOSTIC,    69, -9.058174180355e-01, 3.223001352089e-06, 6.9783e+02, 7.6132e+00,
 1DIAGNOSTIC,    70, -9.058614432038e-01, 3.010262773539e-06, 7.0570e+02, 7.8687e+00,
 1DIAGNOSTIC,    71, -9.059052839097e-01, 2.972786470689e-06, 7.1355e+02, 7.8486e+00,
 1DIAGNOSTIC,    72, -9.058910671336e-01, 2.857625850566e-06, 7.2203e+02, 8.4886e+00,
 1DIAGNOSTIC,    73, -9.059321432803e-01, 2.663670110198e-06, 7.3033e+02, 8.2975e+00,
 1DIAGNOSTIC,    74, -9.059725268402e-01, 2.647944851924e-06, 7.3893e+02, 8.5986e+00,
 1DIAGNOSTIC,    75, -9.059589250103e-01, 2.542843799280e-06, 7.4770e+02, 8.7729e+00,
 1DIAGNOSTIC,    76, -9.060001517814e-01, 2.377403190732e-06, 7.5544e+02, 7.7344e+00,
 1DIAGNOSTIC,    77, -9.060361666088e-01, 2.356599099607e-06, 7.6298e+02, 7.5384e+00,
 1DIAGNOSTIC,    78, -9.060189340520e-01, 2.262225667347e-06, 7.7041e+02, 7.4302e+00,
 1DIAGNOSTIC,    79, -9.060577177445e-01, 2.105318473949e-06, 7.7812e+02, 7.7146e+00,
 1DIAGNOSTIC,    80, -9.060932145621e-01, 2.0947591984

 1DIAGNOSTIC,    61, -9.031831542974e-01, 4.645897919920e-06, 4.3114e+03, 5.7593e+01,
 1DIAGNOSTIC,    62, -9.032511218840e-01, 5.152203636238e-06, 4.3667e+03, 5.5307e+01,
 1DIAGNOSTIC,    63, -9.033254731759e-01, 5.655048703163e-06, 4.4218e+03, 5.5128e+01,
 1DIAGNOSTIC,    64, -9.034079426530e-01, 6.159510300943e-06, 4.4828e+03, 6.0986e+01,
 1DIAGNOSTIC,    65, -9.034957563833e-01, 6.661421093804e-06, 4.5394e+03, 5.6579e+01,
 1DIAGNOSTIC,    66, -9.035888334736e-01, 7.145436507934e-06, 4.5940e+03, 5.4641e+01,
 1DIAGNOSTIC,    67, -9.036871823822e-01, 7.614057994092e-06, 4.6488e+03, 5.4715e+01,
 1DIAGNOSTIC,    68, -9.037850609697e-01, 8.037105263467e-06, 4.7059e+03, 5.7121e+01,
 1DIAGNOSTIC,    69, -9.038827030212e-01, 8.387294830523e-06, 4.7640e+03, 5.8125e+01,
 1DIAGNOSTIC,    70, -9.039788941592e-01, 8.645196027477e-06, 4.8206e+03, 5.6635e+01,
 1DIAGNOSTIC,    71, -9.040753061708e-01, 8.806387155029e-06, 4.8798e+03, 5.9121e+01,
 1DIAGNOSTIC,    72, -9.041249558107e-01, 8.6635227303

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/220617_P201_skeletonization/ants', 'deigo:/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/analysis'], returncode=0)